In [1]:
import os, sys
from pathlib import Path
script_dir = Path(os.path.dirname(os.path.abspath('')))
module_dir = str(script_dir)
sys.path.insert(0, module_dir + '/modules')
print(module_dir)

# import the rest of the modules
%matplotlib nbagg
import numpy as np
import tensorflow as tf 
import matplotlib.pyplot as plt
import sim
import pandas as pd
import tensorflow_probability as tfp
import time  
import math
import utility as ut
tfd = tfp.distributions
seed=42
np.random.seed(seed=seed)
tf.random.set_seed(seed=seed)

C:\Users\pinak\Documents\GitHub\fp-solvers


In [2]:
# set up computation parameters
dim = 3
n_particles = int(1e5)
n_subdivs = 100
save_folder = 'L63-pf'
n_steps = 3
n_repeats = 200
dt = 0.01
alpha, beta, rho = 10., 8./3., 28.
t = dt * n_steps
quad_method = 'Gauss_Legendre'
n_int_subdivs = 100
degree = 6
DTYPE = 'float32'

#### Define $\mu, \sigma, p_0, \sigma_h$

In [3]:
def mu_np(X):
    x, y, z = np.split(X, dim, axis=-1)
    p = alpha * (y - x) 
    q = x * (rho - z) - y 
    r = x * y - beta * z
    return np.concatenate([p, q, r], axis=-1)

sigma = 10.
sigma_h = 5.

l = np.ones(dim, dtype=DTYPE)
g1 = tfd.MultivariateNormalDiag(loc=2.*l, scale_diag=l)
g2 = tfd.MultivariateNormalDiag(loc=-2.*l, scale_diag=l)
mix = 0.5
rv0 = tfd.Mixture(cat=tfd.Categorical(probs=[mix, 1.-mix]), components=[g1, g2])
log_p0 = lambda x: tf.reshape(rv0.log_prob(x), (-1, 1))

#### Define observations

In [4]:
def H(X):
    x, y, z = np.split(X, 3, axis=-1)
    return np.concatenate([x, z], axis=-1)

def obs(X):
    x, y, z = np.split(X, 3, axis=-1)
    x = x + np.random.normal(scale=sigma_h, size=x.shape).astype(DTYPE)
    z = z + np.random.normal(scale=sigma_h, size=z.shape).astype(DTYPE)
    return np.concatenate([x, z], axis=-1)

#### Set up resampling

In [5]:
def get_weights(y, X):
    rv1 = tfd.MultivariateNormalDiag(loc=y, scale_diag=sigma_h*np.ones(len(y)).astype(DTYPE))
    w = rv1.prob(H(X)).numpy()
    return w/w.sum()

@ut.timer
def systematic_noisy_resample(X, weights, resampling_cov=0.1):
        # make N subdivisions, and choose positions with a consistent random offset
        positions = (np.random.random() + np.arange(n_particles)) / n_particles
        indices = np.zeros(n_particles, 'i')
        cumulative_sum = np.cumsum(weights.astype('float64'))
        i, j = 0, 0
       
        while i < n_particles:
            if positions[i] < cumulative_sum[j]:
                indices[i] = j
                i += 1
            else:
                j += 1
        indices = list(set(indices))
        offsprings = [0] * len(indices)
        weight_sum = sum([weights[i] for i in indices])
        for k, i in enumerate(indices):
            offsprings[k] = math.ceil(weights[i]/weight_sum*n_particles)
        new_particles = np.zeros((sum(offsprings), dim))
        mean = np.zeros(dim)
        cov = resampling_cov * np.identity(dim)
        j = 0
        for k, i in enumerate(indices):
            new_particles[j] = X[i]
            new_particles[j+1: j+offsprings[k]]= X[i] + np.random.multivariate_normal(mean, cov, size=offsprings[k] - 1)
            j += offsprings[k]
        particles = np.array([new_particles[i] for i in np.random.choice(sum(offsprings), n_particles, replace=False)])
        return particles

#### Create a true trajectory and observe

In [6]:
true_state = rv0.sample(1).numpy()
for step in range(n_steps):
    true_state +=  mu_np(true_state) * dt + sigma * np.random.normal(scale=np.sqrt(dt), size=(1, dim))
observation = obs(true_state)
np.save('{}/true_state.npy'.format(save_folder), true_state)
np.save('{}/observation.npy'.format(save_folder), observation)

#### Run trajectories

In [7]:
mc = sim.MCProb(save_folder, n_subdivs, mu_np, sigma, rv0.sample(n_particles).numpy())
mc.propagate(n_steps, dt)

Time taken by propagate is 0.26097607612609863 seconds


#### Observe in the end

In [8]:
X = np.genfromtxt('{}/ensemble.csv'.format(mc.save_folder), delimiter=',').astype(DTYPE)
weights = get_weights(observation[0], X)
X_r = systematic_noisy_resample(X, weights, 0.5)
pd.DataFrame(X_r).to_csv('{}/ensemble.csv'.format(mc.save_folder), index=None, header=None)

Time taken by systematic_noisy_resample is 4.7372777462005615 seconds


#### Do box counting

In [9]:
mc.set_grid(lims=None)
mc.assign_pts()
p_1f = mc.compute_p2(0, 1, save=False)
np.save(save_folder + '/p_1f.npy', p_1f)
p_2f = mc.compute_p2(1, 2, save=False)
np.save(save_folder + '/p_2f.npy', p_2f)
p_3f = mc.compute_p2(2, 0, save=False)
np.save(save_folder + '/p_3f.npy', p_3f)

Time taken by set_grid is 0.32370471954345703 seconds
Time taken by assign_pts is 0.3859059810638428 seconds
Time taken by compute_p2 is 0.2953987121582031 seconds
Time taken by compute_p2 is 0.2882242202758789 seconds
Time taken by compute_p2 is 0.286959171295166 seconds


#### Save grid

In [10]:
low = mc.grid.mins
high = mc.grid.maxs
x = np.linspace(low[0], high[0], num=n_subdivs+1)[:-1].astype(DTYPE) + mc.grid.h[0]/2. 
y = np.linspace(low[1], high[1], num=n_subdivs+1)[:-1].astype(DTYPE) + mc.grid.h[1]/2.
z = np.linspace(low[2], high[2], num=n_subdivs+1)[:-1].astype(DTYPE) + mc.grid.h[2]/2.
np.save('{}/x.npy'.format(save_folder), x)
np.save('{}/y.npy'.format(save_folder), y)
np.save('{}/z.npy'.format(save_folder), z)

In [11]:
observation[0][0]

7.796521